In [78]:
import itertools


In [107]:
binding_site = {
    "ASN": [66],
    "GLU": [162, 384, 403, 411],
    "GLN": [281, 530],
    "THR": [282],
    "VAL": [351, 379, 380, 518],
    "HIS": [353, 383, 387, 410, 513],
    "ALA": [354, 356],
    "SER": [355],
    "TRP": [357],
    "ASP": [358, 415, 453],
    "TYR": [360, 394, 520, 523],
    "PHE": [391, 457, 460, 512, 527],
    "GLY": [404],
    "PRO": [407],
    "LYS": [454, 511],
    "ARG": [522],
    "ZN": [701]
}


In [108]:
dif = 0
# add the dif to the binding site
for k, v in binding_site.items():
    binding_site[k] = [x + dif for x in v]

binding_site

{'ASN': [66],
 'GLU': [162, 384, 403, 411],
 'GLN': [281, 530],
 'THR': [282],
 'VAL': [351, 379, 380, 518],
 'HIS': [353, 383, 387, 410, 513],
 'ALA': [354, 356],
 'SER': [355],
 'TRP': [357],
 'ASP': [358, 415, 453],
 'TYR': [360, 394, 520, 523],
 'PHE': [391, 457, 460, 512, 527],
 'GLY': [404],
 'PRO': [407],
 'LYS': [454, 511],
 'ARG': [522],
 'ZN': [701]}

In [109]:
# return the values of the dictionary as a list
def get_binding_site():
    binding_site_list = []
    for k, v in binding_site.items():
        binding_site_list.extend(v)
    return binding_site_list

In [110]:
binding_site_values = get_binding_site()

In [111]:

print(*binding_site_values, sep=",")


66,162,384,403,411,281,530,282,351,379,380,518,353,383,387,410,513,354,356,355,357,358,415,453,360,394,520,523,391,457,460,512,527,404,407,454,511,522,701


In [112]:
binding_site = ",".join(str(v) for v in binding_site_values)
binding_site

'66,162,384,403,411,281,530,282,351,379,380,518,353,383,387,410,513,354,356,355,357,358,415,453,360,394,520,523,391,457,460,512,527,404,407,454,511,522,701'

# Make .icm file to run RMSD

In [115]:
rec = "ace"
path = f"/media/emilia/HDD/Documents/multiple_structures/{rec}/"
pdbs = ["1o8a","1o86", "1uze", "2oc2","3bkl"]

extra_lines = []
# generar_icm.py
for i in pdbs:
    extra_lines.append(f'openFile "{rec}_{i}_rec.mol2"')
icm_lines = [
    '#!/usr/bin/icm',
    'call _startup',
    '',
    'maxFileSizeMb=10000',
]    

icm_lines += extra_lines



# proximo paso: crear arrays para rmsd y hacer superimpose de esos

combinations = list(itertools.combinations(pdbs, 2))
extra_lines = []
for p1, p2 in combinations:
    icm_lines.append(f'rmsd_dock_{p1}_{p2}=Rarray(1,0.)')
    idx1 = pdbs.index(p1) + 1
    idx2 = pdbs.index(p2) + 1
    extra_lines.append(
        f'superimpose a_{idx1}.*/SHGEI/ca,c,n a_{idx2}.*/SHGEI/ca,c,n align'
    )
    # proximo paso: calc RMSD
    extra_lines.append(
        f'rmsd_dock_{p1}_{p2}[1]= Srmsd(a_{idx1}.*/{binding_site}/!h* a_{idx2}.*/{binding_site}/!h* chemical)'
    )


icm_lines += extra_lines


# Escribir archivo

with open(path + f"{rec}_analysis.icm", "w") as f:
    for line in icm_lines:
        f.write(line + "\n")

print("Archivo .icm generado como 'fabp4_analysis.icm'")



Archivo .icm generado como 'fabp4_analysis.icm'


In [102]:
print(icm_lines)

['#!/usr/bin/icm', 'call _startup', '', 'maxFileSizeMb=10000', 'openFile "emi_a_rec.mol2"', 'openFile "emi_b_rec.mol2"', 'rmsd_dock_a_b=Rarray(1,0.)', 'superimpose a_1.*/SHGEI/ca,c,n a_2.*/SHGEI/ca,c,n align', 'rmsd_dock_a_b= Srmsd a_1.*/77,78,79,80,156,30,82,105,32,220,221,222,223,224,225,226,298,34,37,310,35,36,39,40,41,112,300,51,116,308,311,55,295,297,75,218,76,108,113,118,129,130,131,227,303,306,12/!h* a_2.*/77,78,79,80,156,30,82,105,32,220,221,222,223,224,225,226,298,34,37,310,35,36,39,40,41,112,300,51,116,308,311,55,295,297,75,218,76,108,113,118,129,130,131,227,303,306,12/!h* chemical']
